In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from functools import reduce
from pandas.plotting import scatter_matrix, lag_plot, autocorrelation_plot

%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def get_data(token):
    if token == "FLOW":
        res = requests.get(f'https://api.cryptowat.ch/markets/kraken/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )       
    else:
        res = requests.get(
        f'https://api.cryptowat.ch/markets/coinbase-pro/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [3]:
tokens = ['ETH', 'SOL', 'AVAX', 'USDT', 'FLOW']

In [4]:
df_base = pd.concat(get_data(token) for token in tokens)
df_baseVOLUME=df_base.groupby('token')['volumeUSD'].sum().to_frame()
df_baseVOLUME.sort_values('volumeUSD',ascending=False)

,volumeUSD
token,
ETH,3.057970e+10
SOL,1.027092e+10
AVAX,5.865315e+09
USDT,2.856511e+09
FLOW,1.108827e+08


In [5]:
df_base

,ts,open,high,low,close,volume,volumeUSD,token
0,2021-11-22 00:00:00,4317.980,4342.240,4246.070,4262.990,7262.562789,3.123077e+07,ETH
1,2021-11-22 01:00:00,4263.040,4270.340,4212.450,4234.370,8437.946084,3.574586e+07,ETH
2,2021-11-22 02:00:00,4234.870,4246.720,4171.170,4217.890,9259.725370,3.895639e+07,ETH
3,2021-11-22 03:00:00,4217.880,4223.480,4163.580,4193.470,9259.899519,3.882032e+07,ETH
4,2021-11-22 04:00:00,4192.950,4213.590,4147.000,4168.350,7934.546906,3.315476e+07,ETH
...,...,...,...,...,...,...,...,...
915,2021-12-30 03:00:00,8.552,8.552,8.518,8.530,1381.801600,1.179145e+04,FLOW
916,2021-12-30 04:00:00,8.554,8.554,8.517,8.531,722.358451,6.177095e+03,FLOW
917,2021-12-30 05:00:00,8.517,8.570,8.517,8.565,1361.430014,1.164108e+04,FLOW
918,2021-12-30 06:00:00,8.557,8.576,8.547,8.565,3056.162767,2.615952e+04,FLOW


In [6]:
df_base['testValue']=df_base.volume/df_base.volumeUSD

In [26]:
(df_base.close - df_base.close.cummax())/df_base.close

0        0.000000
1       -0.006759
2       -0.010693
3       -0.016578
4       -0.022704
          ...    
915   -558.507620
916   -558.442035
917   -556.221249
918   -556.221249
919   -552.665893
Name: close, Length: 4600, dtype: float64

In [19]:
x=df_base.volume/df_base.volume.shift(1)

In [25]:

df_base.close.cummax()

0      4262.99
1      4262.99
2      4262.99
3      4262.99
4      4262.99
        ...   
915    4772.60
916    4772.60
917    4772.60
918    4772.60
919    4772.60
Name: close, Length: 4600, dtype: float64